In [27]:
import pandas as pd 

from difflib import SequenceMatcher
from rapidfuzz import process, fuzz
from difflib import SequenceMatcher

In [31]:
!ls ../cc/

companies.csv  countries.csv  industries.csv


In [356]:
sbti_df =  pd.read_excel('../target-dashboard.xlsx')
companies_df = pd.read_csv('../cc/companies.csv')         # <-- 
industries_df = pd.read_csv('../cc/industries.csv')       # <-- AppIndustries table from PRD DB
countries_df = pd.read_csv('../cc/countries.csv')         # <--

## Ignore companies with removed targets

In [357]:
sbti_df[sbti_df['Status']=='Removed']['Company Name'].unique()

array(['3B-Fibreglass', 'ACCEDO BROADBAND AB', 'Accor S.A.',
       'Ace & Tate Holding B.V.', 'Ace Technologies Co.,Ltd.', 'Acensi',
       'Acerinox,S.A.', 'AÇO VERDE DO BRASIL.S.A',
       'ACROX TECHNOLOGIES CO., LTD.', 'ADEN SERVICES CHINA (GROUP)',
       'Adva Optical Networking SE', 'Aegon Nederland N.V.',
       'AFRY (ÅF Pöyry)', 'Å Energi', 'Agility', 'Agropur Cooperative',
       'Aico',
       'Airsys Refrigeration Engineering Technology (Beijing) Co., Ltd.',
       'Aitken Spence PLC', 'AJE GROUP', 'Akamai Technologies INC',
       'AKDAŞ DÖKÜM A.Ş.', 'Akila',
       'Albert Bartlett and Sons (Airdrie) Ltd.',
       'Alchem International Private Limited', 'ALD',
       'ALE INTERNATIONAL SASU (GROUP)',
       'Alkaram Towel Industries (Pvt.) Ltd.',
       'Alliance One Apparel Co.,Ltd', 'Allianz Investment Management SE',
       'Alltech, Inc.', 'ALPHA INDO NUSA', 'Alpin Çorap San. Ve Tic. A.Ş',
       'Alun Griffiths (Contractors) Ltd', 'Amalgamated Construction Ltd',
  

In [358]:
companies_with_removed_targets = sbti_df[sbti_df['Status']=='Removed']['Company Name'].unique()
sbti_df = sbti_df[~sbti_df['Company Name'].isin(companies_with_removed_targets)]


## Task Description

SBTi uploading task is about 
1. Uploading the tenants, which exists are in SBTi, but not in CC'DB.
2. Adding targets of this tenants

## Find a list of companies TBA

* Company name can be written in different way for example
  In SBTi dashboard "Cargill, Inc."           ->.       In the platform "Cargill".
* This section is about find a list of companies to be added to the platform, taking into account the difficutlies of spelling. 

In [359]:
sbti_companies = sbti_df['Company Name'].str.strip().dropna().str.lower()
tenants_companies_corrected = companies_df['Name'].str.strip().dropna().str.lower()

def preprocess_company_name(name):
    #if not isinstance(name, str):
    #  return

    # Remove common suffixes
    suffixes = ['inc', 'llc', 'corp', 'corporation', 'company', 'co', 'limited', 'ltd', '.', ',']
    for suffix in suffixes:
        name = name.replace(suffix, '')
    return name.strip()

# Apply preprocessing to the company names
sbti_companies = sbti_companies.apply(preprocess_company_name)
tenants_companies_corrected = tenants_companies_corrected.apply(preprocess_company_name)


In [360]:

def find_similar_names_rapidfuzz(list1, list2, threshold=90):
    matches = []
    i = 0
    for name1 in list1:
        if i % 100 == 0:
          print(i)

        i = i + 1
        best_match = process.extractOne(name1, list2, scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] >= threshold:
            matches.append((name1, best_match[0]))
            print((name1, best_match[0]))
    return matches

# Find similar names
matches_rapidfuzz = find_similar_names_rapidfuzz(sbti_companies, tenants_companies_corrected)

0
('deutsche glasfaser group gmbh', 'deutsche glasfaser group gmbh')
('drax group plc', 'drax group plc')
('ab inbev', 'ab inbev')
('abbvie', 'abbvie')
('acbel polytech', 'acbel polytech')
('accenture', 'accenture')
('archer daniels midland', 'archer daniels midland')
('aigle international sa', 'aigle international')
('allied sustainability and environmental nsultants group', 'allied sustainability and environmental nsultan')
100
('api restauration', 'api restauration')
('atlassian oration plc', 'atlassian oration plc')
('aubay', 'aubay')
200
('bce', 'bce')
('berner', 'berner')
300
('brenntag se', 'brenntag sa')
('bureau veritas', 'bureau veritas')
400
('chalhoub group', 'chalhoub group')
('citrosu agroindustria sa', 'citrosu agroindustria sa')
('cnp assurances', 'cnp assurances')
500
('danone', 'danone')
('decathlon', 'decathlon')
600
('döhler group se', 'döhler group se')
('drax group plc', 'drax group plc')
('drt (a  of the firmenich group)', 'drt (a  of the firmenich group)')
('ela

filtered_sbti_df contained the companies, TBA

In [361]:
# Create a DataFrame for the matching companies
matches_rapidfuzz_df = pd.DataFrame(matches_rapidfuzz, columns=['SBTI Company Name', 'Tenant Company Name'])
cleaned_company_names = sbti_df['Company Name'].str.strip().dropna().str.lower()
sbti_df['cleaned_company_name'] = sbti_df['Company Name'].str.strip().str.lower().apply(preprocess_company_name)

# Step 2: Convert the 'SBTI Company Name' column in matches_rapidfuzz_df to lowercase
matches_rapidfuzz_company_names = matches_rapidfuzz_df['SBTI Company Name'].str.lower()

# Step 3: Filter sbti_df based on whether the cleaned company names exist in matches_rapidfuzz_company_names
filtered_sbti_df = sbti_df[~sbti_df['cleaned_company_name'].isin(matches_rapidfuzz_company_names)]


In [362]:
filtered_sbti_df[sbti_df['Company Name'] == 'PepsiCo, Inc.']

/var/folders/73/qg89nk6105d6084yrk56sfg00000gn/T/ipykernel_22088/711860501.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_sbti_df[sbti_df['Company Name'] == 'PepsiCo, Inc.']


,Company Name,ISIN,LEI,Location,Region,Sector,Organization Type,Action,Commitment Type,Commitment Deadline,...,Scope,Target Value,Type,Sub-type,Target Classification,Base Year,Target Year,Year Type,Date Published,cleaned_company_name


## Map Countries

In [363]:
sbti_countries = filtered_sbti_df['Location'].unique()
cc_countries = countries_df.Name

In [364]:
countries_map_df = pd.DataFrame({
    'sbti': sbti_countries,
    'cc': sbti_countries
})

In [365]:
not_matching_countries = set(sbti_countries) - set(cc_countries)
not_matching_countries

{'Bosnia-Herzegovina',
 'Czechia',
 'Hong Kong, China',
 'Korea, Republic of',
 'Myanmar',
 'Philippines',
 'Russian Federation',
 'Taiwan, Province of China',
 'Tanzania, United Republic of',
 'Türkiye',
 'United States of America',
 'Viet Nam'}

In [366]:
countries_map_df[countries_map_df.sbti == 'Czechia']

,sbti,cc
45,Czechia,Czechia


In [367]:
countries_map_df.loc[countries_map_df['sbti'] == 'Czechia', 'cc'] = 'Czech Republic'
countries_map_df.loc[countries_map_df['sbti'] == 'Bosnia-Herzegovina', 'cc'] = 'Bosnia and Herzegovina'
countries_map_df.loc[countries_map_df['sbti'] == 'Hong Kong, China', 'cc'] = 'Hong Kong'
countries_map_df.loc[countries_map_df['sbti'] == 'Korea, Republic of', 'cc'] = 'South Korea'
# Remove Macau <-- doesn't exist on our platform
countries_map_df.loc[countries_map_df['sbti'] == 'Macau, China', 'cc'] = None
countries_map_df.loc[countries_map_df['sbti'] == 'Myanmar', 'cc'] = 'Myanmar (Burma)'
# Remove Philippines
countries_map_df.loc[countries_map_df['sbti'] == 'Philippines', 'cc'] = None
countries_map_df.loc[countries_map_df['sbti'] == 'Russian Federation', 'cc'] = 'Russia'
countries_map_df.loc[countries_map_df['sbti'] == 'Taiwan, Province of China', 'cc'] = 'Taiwan'
countries_map_df.loc[countries_map_df['sbti'] == 'Tanzania, United Republic of', 'cc'] = 'Tanzania'
countries_map_df.loc[countries_map_df['sbti'] == 'Türkiye', 'cc'] = 'Turkey'
countries_map_df.loc[countries_map_df['sbti'] == 'United States of America', 'cc'] = 'United States'
countries_map_df.loc[countries_map_df['sbti'] == 'Viet Nam', 'cc'] = 'Vietnam'



In [368]:
filtered_sbti_df.columns

Index(['Company Name', 'ISIN', 'LEI', 'Location', 'Region', 'Sector',
       'Organization Type', 'Action', 'Commitment Type', 'Commitment Deadline',
       'Status', 'Reason for commitment extension or removal',
       'Full target language', 'Company Temperature Alignment', 'Target',
       'Target Wording', 'Scope', 'Target Value', 'Type', 'Sub-type',
       'Target Classification', 'Base Year', 'Target Year', 'Year Type',
       'Date Published', 'cleaned_company_name'],
      dtype='object')

In [369]:
mapping = dict(zip(countries_map_df['sbti'], countries_map_df['cc']))
filtered_sbti_df['Location'] = filtered_sbti_df['Location'].map(mapping)
filtered_sbti_df = filtered_sbti_df.dropna(subset=['Location'])

/var/folders/73/qg89nk6105d6084yrk56sfg00000gn/T/ipykernel_22088/3035417504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_sbti_df['Location'] = filtered_sbti_df['Location'].map(mapping)


In [370]:
sbti_countries = filtered_sbti_df['Location'].unique()
cc_countries = countries_df.Name
not_matching_countries = set(sbti_countries) - set(cc_countries)
not_matching_countries

set()

## Map Industries

In [371]:
sbti_industries = filtered_sbti_df['Sector'].unique()
cc_industries = industries_df.Name

In [372]:
set(sbti_industries) - set(cc_industries)

{'Electric Utilities and Independent Power Producers and Energy Traders (including Fossil, Alternative and Nuclear Energy)',
 'Gas Utilities',
 nan}

In [373]:
industries_map_df = pd.DataFrame({
    'sbti': sbti_industries,
    'cc': sbti_industries
})

In [374]:
industries_map_df.loc[industries_map_df['sbti'] == 'Electric Utilities and Independent Power Producers and Energy Traders (including Fossil, Alternative and Nuclear Energy)', 'cc'] = 'Electric Utilities and Independent Power Producers and Energy Traders (including fossil, alternative and nuclear energy)'
industries_map_df.loc[industries_map_df['sbti'] == 'Gas Utilities', 'cc'] = None
industries_map_df.loc[industries_map_df['sbti'] == 'nan', 'cc'] = None

In [375]:
mapping = dict(zip(industries_map_df['sbti'], industries_map_df['cc']))
filtered_sbti_df['Sector'] = filtered_sbti_df['Sector'].map(mapping)
filtered_sbti_df = filtered_sbti_df.dropna(subset=['Sector'])

In [376]:
sbti_industries = filtered_sbti_df['Sector'].unique()
cc_industries = industries_df.Name
set(sbti_industries) - set(cc_industries)

set()

## Add Country and Industry Ids

In [377]:
countries_df.head()

,Id,Name,TwoCharCode,ThreeCharCode,Name.1,TwoCharCode.1,ThreeCharCode.1
0,1,Afghanistan,AF,AFG,Afghanistan,AF,AFG
1,2,Aland Islands,AX,ALA,Aland Islands,AX,ALA
2,3,Albania,AL,ALB,Albania,AL,ALB
3,4,Algeria,DZ,DZA,Algeria,DZ,DZA
4,5,American Samoa,AS,ASM,American Samoa,AS,ASM


In [378]:
mapping = dict(zip(countries_df.Name, countries_df.Id))
filtered_sbti_df['CountryId'] = filtered_sbti_df['Location'].map(mapping)
filtered_sbti_df.CountryId.isna().any()

np.False_

In [379]:
mapping = dict(zip(industries_df.Name, industries_df.Id))
filtered_sbti_df['IndustryId'] = filtered_sbti_df['Sector'].map(mapping)
filtered_sbti_df.IndustryId.isna().any()

np.False_

## Save the results

We are interested only in CompanyName, CuntryId, SectorId

In [380]:
result_df = filtered_sbti_df[[
    'Company Name',
    'CountryId',
    'IndustryId'
]].drop_duplicates(subset=['Company Name'])


In [381]:
result_df.to_csv('companies_to_upload.csv', index=False)